In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba 
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
import multiprocessing
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split 
from sklearn.utils import shuffle 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
%matplotlib inline

D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
# set parameters(word2vec)
vocab_dim = 200
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
cpu_count = multiprocessing.cpu_count()

In [3]:
data=pd.read_csv('data/train_first.csv')
#data=data.drop(['Id'],axis=1)
#data.head()
data=data.sample(frac=1).reset_index(drop=True) 

#划分x  y
X=data['Discuss']
Y=data['Score']
 
#y ONEHOT
lb = preprocessing.LabelBinarizer()
lb.fit([1,2,3,4,5])
Y=lb.transform(Y)

In [4]:
X[0]

'了之后才知道为啥这个景区那么有名，一定要先了解下历史。可以说，有了这个都江堰，才有后来土地如此肥沃的天府之国，四川人民都应该感谢李冰。工程比较宏伟，门票价格比较贵100元每人。夏天去站在桥上比较凉快，挨着古城，可以去买吃买喝，价格不贵！节假日人很多！停车10元'

In [5]:
#加载停用词
def get_stopwords(path):
    return [line.strip() for line in open(path,'r',encoding='utf-8').readlines()]
#句子去停用词
def removestopwords(sentence):
        stopwords_list=get_stopwords('data/stopwords.txt')
        outstr=[]
        for word in sentence:
            if not word in stopwords_list:
                if word!='\n' and word!='\t':
                     outstr.append(word)
        return outstr

In [6]:
#分词 并去掉停用词
def cut(sentence):
    return removestopwords(jieba.cut(sentence))
#分词后的word
cabs=[cut(x) for x in X]
#cabs[0]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhanggd\AppData\Local\Temp\5\jieba.cache
Loading model cost 3.049 seconds.
Prefix dict has been built succesfully.


In [7]:
#word2vec

#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)
    model.train(combined,total_examples=model.corpus_count,epochs=model.iter)
    model.save('data/Word2vec.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量

        def parse_dataset(combined):
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')

In [8]:
#根据词向量重新填充x
index_dict, word_vectors,combined=word2vec_train(cabs)


D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':
D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [9]:
#getdata
# def get_data(index_dict,word_vectors,combined,y):
#     n_symbols = len(index_dict) + 1  # 所有单词的索引数，频数小于10的词语索引为0，所以加1
#     embedding_weights = np.zeros((n_symbols, vocab_dim))#索引为0的词语，词向量全为0
#     for word, index in index_dict.items():#从索引为1的词语开始，对每个词语对应其词向量
#         embedding_weights[index, :] = word_vectors[word]
#     x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
#     print (x_train.shape,y_train.shape)
#     return n_symbols,embedding_weights,x_train,y_train,x_test,y_test

In [10]:
#搭建网络结构
input_x=tf.placeholder(shape=[None,None],dtype=tf.int32,name='input_x')
label=tf.placeholder(shape=[None,None],dtype=tf.int32,name='target')
lr=tf.placeholder(dtype=tf.float32,name='learning_rate')
#sequence_lengths=tf.placeholder(shape=[None],dtype=tf.int32,name='sequence_lengths')
keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')
#embedding_placeholder = tf.placeholder(tf.float32, [None, None],name='embedding_placeholder')
#super params
hidden_dim=128
batch_size=100
layer_num=2
#bilstm
# with tf.variable_scope('bi_lstm'):
#     #lstm层
#     lstm_fw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     lstm_bw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     #dropout
#     lstm_fw_cell=rnn.DropoutWrapper(cell=lstm_fw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     lstm_bw_cell=rnn.DropoutWrapper(cell=lstm_bw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     #多层lstm
#     cell_fw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     cell_bw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     #初始状态
#     initial_state_fw=cell_fw.zero_state(batch_size,tf.float32)
#     initial_state_bw=cell_bw.zero_state(batch_size,tf.float32)


#with tf.variable_scope('embedding'):
#embedding = tf.Variable(tf.constant(0.0, shape=[len(index_dict)+1,vocab_dim]),trainable=True, name="embedding")
embedding = tf.Variable(tf.random_uniform((len(index_dict)+1,vocab_dim), -1, 1),name='embedding')
#tf.assign(embedding,embedding_placeholder)
#print(embedding)
embed = tf.nn.embedding_lookup(embedding, input_x)
#lstm
#with tf.variable_scope('lstm'):

def get_lstm_dropout():
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(hidden_dim)
    return tf.contrib.rnn.DropoutWrapper(cell=lstm_cell,output_keep_prob=keep_prob)
cell=tf.contrib.rnn.MultiRNNCell([get_lstm_dropout() for _ in range(layer_num)] )
initial_state=cell.zero_state(batch_size,tf.float32)
outputs, final_state=tf.nn.dynamic_rnn(cell,embed,initial_state=initial_state)

#with tf.variable_scope('optmizer'):
predictions=tf.contrib.layers.fully_connected(outputs[:,-1],5,activation_fn=tf.nn.softmax)
cost=tf.losses.mean_squared_error(label,predictions)
optimizer=tf.train.AdamOptimizer(lr).minimize(cost)
#validation accuracy
correct_pred=tf.equal(tf.cast(tf.round(predictions),tf.int32),label)
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [11]:
#batching
def get_batching(x,y,batch_size=100):
    n_batches=len(x)//batch_size
    x,y=x[:n_batches*batch_size],y[:n_batches*batch_size]
    for ii in range(0,len(x),batch_size):
        yield x[ii:ii+batch_size],y[ii:ii+batch_size]        

In [12]:
#训练
#super params
epochs=10
saver=tf.train.Saver()
with tf.Session()as sess:
    writer = tf.summary.FileWriter("logs/", sess.graph)
    sess.run(tf.global_variables_initializer())
    x_train, x_test, y_train, y_test=train_test_split(combined, Y, test_size=0.2)#get_data(index_dict,word_vectors,combined,Y)
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    iteration = 1
    for e in range(epochs):
        for ii, (x, y) in enumerate(get_batching(x_train, y_train, batch_size), 1):
            feed={input_x:x,label:y,lr:0.001,keep_prob:0.5}
            loss,_=sess.run([cost,optimizer],feed_dict=feed)
            tf.summary.scalar('loss',loss)
            if iteration%50==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            if iteration%100==0:
                val_acc = []
                for x, y in get_batching(x_test, y_test, batch_size):
                    feed = {input_x: x,
                            label: y,                            
                            keep_prob: 1}
                    batch_acc= sess.run(accuracy, feed_dict=feed)
                    val_acc.append(batch_acc)
                tf.summary.scalar('accuracy',np.mean(val_acc))
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

(80000, 100)
(80000, 5)
(20000, 100)
(20000, 5)
Epoch: 0/10 Iteration: 50 Train loss: 0.115
Epoch: 0/10 Iteration: 100 Train loss: 0.102
Val acc: 0.839
Epoch: 0/10 Iteration: 150 Train loss: 0.097
Epoch: 0/10 Iteration: 200 Train loss: 0.105
Val acc: 0.846
Epoch: 0/10 Iteration: 250 Train loss: 0.095
Epoch: 0/10 Iteration: 300 Train loss: 0.090
Val acc: 0.848
Epoch: 0/10 Iteration: 350 Train loss: 0.108
Epoch: 0/10 Iteration: 400 Train loss: 0.096
Val acc: 0.851
Epoch: 0/10 Iteration: 450 Train loss: 0.084
Epoch: 0/10 Iteration: 500 Train loss: 0.080
Val acc: 0.852
Epoch: 0/10 Iteration: 550 Train loss: 0.099
Epoch: 0/10 Iteration: 600 Train loss: 0.104
Val acc: 0.853
Epoch: 0/10 Iteration: 650 Train loss: 0.102
Epoch: 0/10 Iteration: 700 Train loss: 0.104
Val acc: 0.854
Epoch: 0/10 Iteration: 750 Train loss: 0.106
Epoch: 0/10 Iteration: 800 Train loss: 0.078
Val acc: 0.855
Epoch: 1/10 Iteration: 850 Train loss: 0.104
Epoch: 1/10 Iteration: 900 Train loss: 0.093
Val acc: 0.855
Epoch: 1

Epoch: 9/10 Iteration: 7700 Train loss: 0.040
Val acc: 0.852
Epoch: 9/10 Iteration: 7750 Train loss: 0.055
Epoch: 9/10 Iteration: 7800 Train loss: 0.048
Val acc: 0.850
Epoch: 9/10 Iteration: 7850 Train loss: 0.065
Epoch: 9/10 Iteration: 7900 Train loss: 0.063
Val acc: 0.849
Epoch: 9/10 Iteration: 7950 Train loss: 0.050
Epoch: 9/10 Iteration: 8000 Train loss: 0.046
Val acc: 0.849


In [13]:
predict_data=pd.read_csv('data/predict_first.csv')
cabs=[cut(x) for x in predict_data['Discuss']]
_,_,predict_X=word2vec_train(cabs)

D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':
D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [21]:
saver=tf.train.Saver()
model_file=tf.train.latest_checkpoint('checkpoints/')
with tf.Session()as sess:
    #sess.run(tf.global_variables_initializer()) 
    saver.restore(sess,model_file)
    predict_arr=[]
    for ii in range(0,len(predict_X),100):
        for p in sess.run(predictions,feed_dict={input_x:predict_X[ii:ii+100],keep_prob:1.0}):
            predict_arr.append(np.argmax(p)+1)
    #print(sess.run(predictions,feed_dict={input_x:predict_X[0:100],keep_prob:1.}))
    

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
